以下を参考にCosmos DBのPython SDKを使って基本的な操作を行う。

https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/cosmos/azure-cosmos/README.md

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from azure.cosmos import CosmosClient

import os
URL = os.environ['ACCOUNT_URI']
KEY = os.environ['ACCOUNT_KEY']
client = CosmosClient(URL, credential=KEY)

In [3]:
from azure.cosmos import exceptions

DATABASE_NAME = 'testDatabase'
try:
    database = client.create_database(DATABASE_NAME)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(DATABASE_NAME)

database

<DatabaseProxy [dbs/testDatabase]>

In [5]:
from azure.cosmos import PartitionKey

database = client.get_database_client(DATABASE_NAME)
CONTAINER_NAME = 'products'

try:
    container = database.create_container(id=CONTAINER_NAME, partition_key=PartitionKey(path="/productName"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
except exceptions.CosmosHttpResponseError:
    raise

container

<ContainerProxy [dbs/testDatabase/colls/products]>

In [6]:
for i in range(1, 10):
    container.upsert_item({
            'id': 'item{0}'.format(i),
            'productName': 'Widget',
            'productModel': 'Model {0}'.format(i)
        }
    )


In [9]:
import json

for item in container.query_items(
        query=f'SELECT * FROM {CONTAINER_NAME}',
        enable_cross_partition_query=True):
    print(json.dumps(item, indent=True))


{
 "id": "item1",
 "productName": "Widget",
 "productModel": "Model 1",
 "_rid": "LNV1AIlaHwsBAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsBAAAAAAAAAA==/",
 "_etag": "\"63003642-0000-2300-0000-66b331ae0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019694
}
{
 "id": "item2",
 "productName": "Widget",
 "productModel": "Model 2",
 "_rid": "LNV1AIlaHwsCAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsCAAAAAAAAAA==/",
 "_etag": "\"63003742-0000-2300-0000-66b331ae0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019694
}
{
 "id": "item3",
 "productName": "Widget",
 "productModel": "Model 3",
 "_rid": "LNV1AIlaHwsDAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsDAAAAAAAAAA==/",
 "_etag": "\"63003842-0000-2300-0000-66b331af0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019695
}
{
 "id": "item4",
 "productName": "Widget",
 "productModel": "Model 4",
 "_rid": "LNV1AIlaHwsEAAAAAAAAAA==",
 "_self": "d

In [10]:
for item in container.query_items(
        query='SELECT * FROM products p WHERE p.productModel = "Model 2"',
        enable_cross_partition_query=True):
    print(json.dumps(item, indent=True))
    container.delete_item(item, partition_key='Widget')


{
 "id": "item2",
 "productName": "Widget",
 "productModel": "Model 2",
 "_rid": "LNV1AIlaHwsCAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsCAAAAAAAAAA==/",
 "_etag": "\"63003742-0000-2300-0000-66b331ae0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019694
}


In [11]:
for item in container.read_all_items():
    print(json.dumps(item, indent=True))

{
 "id": "item1",
 "productName": "Widget",
 "productModel": "Model 1",
 "_rid": "LNV1AIlaHwsBAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsBAAAAAAAAAA==/",
 "_etag": "\"63003642-0000-2300-0000-66b331ae0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019694
}
{
 "id": "item3",
 "productName": "Widget",
 "productModel": "Model 3",
 "_rid": "LNV1AIlaHwsDAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsDAAAAAAAAAA==/",
 "_etag": "\"63003842-0000-2300-0000-66b331af0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019695
}
{
 "id": "item4",
 "productName": "Widget",
 "productModel": "Model 4",
 "_rid": "LNV1AIlaHwsEAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsEAAAAAAAAAA==/",
 "_etag": "\"63003942-0000-2300-0000-66b331af0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019695
}
{
 "id": "item5",
 "productName": "Widget",
 "productModel": "Model 5",
 "_rid": "LNV1AIlaHwsFAAAAAAAAAA==",
 "_self": "d

In [12]:
for item in container.query_items(
        query=f'SELECT * FROM {CONTAINER_NAME} r WHERE r.id="item3"',
        enable_cross_partition_query=True):
    print(json.dumps(item, indent=True))

{
 "id": "item3",
 "productName": "Widget",
 "productModel": "Model 3",
 "_rid": "LNV1AIlaHwsDAAAAAAAAAA==",
 "_self": "dbs/LNV1AA==/colls/LNV1AIlaHws=/docs/LNV1AIlaHwsDAAAAAAAAAA==/",
 "_etag": "\"63003842-0000-2300-0000-66b331af0000\"",
 "_attachments": "attachments/",
 "_ts": 1723019695
}
